# Designing Networks

Let's design some modern networks for computer vision and sequence modeling. 

In [ ]:
import d2l
import numpy as np
import mxnet as mx
from mxnet import nd, autograd, gluon
from mxnet.gluon import nn
from matplotlib import pyplot as plt

ctx = d2l.try_gpu()

## Building Blocks

As a warmup we implement LeNet.

In [ ]:
lenet = nn.Sequential()

lenet.add(
    nn.Conv2D(channels=6, kernel_size=5, activation='relu'),
    nn.MaxPool2D(pool_size=2, strides=2),
    nn.Conv2D(channels=16, kernel_size=3, activation='relu'),
    nn.MaxPool2D(pool_size=2, strides=2),
    nn.Flatten(),
    nn.Dense(120, activation="relu"),
    nn.Dense(84, activation="relu"),
    nn.Dense(10))

In [ ]:
lenet

## Hybridization (JIT compiler)

* Python is slow, compiled backend is fast
* Invoke Python *once* and then fix the graph
* `HybridBlock` is a block that can be JIT compiled

In [ ]:
lenet.hybridize(static_alloc=True, static_shape=True)

``` python
class HybridNet(gluon.HybridBlock):
    def forward(self, F, x):
        # Computation based on other `HybridBlock`
```

In [ ]:
lenet = nn.HybridSequential()
lenet.add(
    nn.Conv2D(channels=6, kernel_size=5, activation='relu'),
    nn.MaxPool2D(pool_size=2, strides=2),
    nn.Conv2D(channels=16, kernel_size=3, activation='relu'),
    nn.MaxPool2D(pool_size=2, strides=2),
    nn.Flatten(),
    nn.Dense(120, activation="relu"),
    nn.Dense(84, activation="relu"),
    nn.Dense(10))
lenet.hybridize(static_alloc=True, static_shape=True)

lenet

### Initialization binds weight vectors

In [ ]:
lenet.initialize(ctx=ctx) #(batch_size, color_channels, height, width)
x = nd.random.uniform(shape=(4,1,28,28), ctx=ctx)
y = lenet(x)
y.shape

In [ ]:
lenet

### MNIST

In [ ]:
def normalize_and_copy(dataset, ctx):
    data = mx.nd.array(dataset._data, ctx=ctx) \
                .transpose((0, 3, 1, 2)) \
                .astype(np.float32)/255
    label = dataset._label
    output = gluon.data.ArrayDataset(data, label)
    output._data[1] = mx.nd.array(label, ctx=ctx)
    return output

In [ ]:
train_data = normalize_and_copy(gluon.data.vision.FashionMNIST(train=True), ctx)
test_data = normalize_and_copy(gluon.data.vision.FashionMNIST(train=False), ctx)

batch_size = 64
train_batches = mx.gluon.data.DataLoader(train_data, batch_size, shuffle=True)
test_batches = mx.gluon.data.DataLoader(test_data, batch_size, shuffle=False)

In [ ]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(lenet.collect_params(), 'sgd', {'learning_rate': .02})

### Evaluation

In [ ]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    for i, (data, label) in enumerate(data_iterator):
        prediction = net(data).argmax(axis=-1, keepdims=True)
        acc.update(preds=prediction, labels=label)
    return acc.get()[1]

### Training

In [ ]:
num_epochs = 10
animator = d2l.Animator(xlabel='epoch', ylabel='accuracy',
                            legend=['train loss', 'train', 'test'], xlim=[0, num_epochs])
for e in range(num_epochs):
    cumulative_loss = 0
    for i, (data, label) in enumerate(train_batches):
        with autograd.record():
            output = lenet(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(data.shape[0])
        cumulative_loss += nd.sum(loss)

    train_accuracy = evaluate_accuracy(train_batches, lenet)
    test_accuracy = evaluate_accuracy(test_batches, lenet)
    print("Epoch %s. Loss: %.4f, Train_acc %.4f, Test_acc %.4f" %
          (e+1, cumulative_loss.asscalar()/len(train_data), train_accuracy, test_accuracy))
    animator.add(e+1, [cumulative_loss.asscalar()/len(train_data), train_accuracy, test_accuracy])

## Recurrent Neural Networks


$$p(x) = \prod_{i=1}^T p(x_{i+1}|x_{[1:i]}) \approx \prod_{i=1}^T p(x_{i+1}|h(x_{[1:i]})) = 
\prod_{i=1}^T p(x_{i+1}|h(x_i, h_{i-1}))
$$

<img src="../img/rnn_unfold.png">

More details of [multilayer perceptrons](https://d2l.ai/chapter_multilayer-perceptrons/index.html), [convolutional neural networks](https://d2l.ai/chapter_convolutional-neural-networks/index.html) and [recurrent neural networks](https://d2l.ai/chapter_recurrent-neural-networks/index.html) are described in [Dive into Deep Learning](https://d2l.ai).